In [2]:
from postgres_connect import *
import requests
import logging

Getting Data

In [6]:
query = '''
select r.rider_id, node_node_id, rider_pip.rider_name, rider_pip.hub, rider_pip.shipping_city as city, r.locus_rider_id
from rider_pip 
left join application_db.rider as r on r.locus_rider_id = rider_pip.rider_id
where 
        pip_status = 'closed' 
and     pip_result = 'failed'
and     pip_date in ('2024-03-04','2024-03-11')
order by node_node_id
'''

query2 = '''
select 
    rider_id, node_node_id
from application_db.rider
where locus_rider_id in (
'Gourav_8527634900_020',
'kaushal_9793911790_020',
'vikash_8954320179_020',
'AJAD_9873736031_011',
'Deepak_kumar_20',
'nikhil_9263902787_020',
'ujjawal__20',
'Deep_9911771714_020',
'sunil_ram_20',
'Sandeep_Kumar__20',
'Darshan_8595785494_020',
'Deepak_8826262741_020',
'Jasveer_Singh_20',
'Praveen_kumar_20',
'Haricharan_yadav_20',
'Sourabh_9911894322_020',
'Manoj_8076388056_020',
'Ajay_kumar_20',
'PRINCE_9654005272_020',
'Yogesh_kumar_20',
'Rohit_8510903003_020',
'Aashish_nagar_20',
'Harsh_kumar_20',
'ankur_8368060526_020',
'ranjeet_7838421627_020',
'RAM_9528858863_020',
'Ravi_7838457674_020',
'Abhishek_chaturvedi__20',
'karan_20',
'sushant__20',
'Faheem_20',
'Ajay_6397825082_020',
'priyanshu_20_1700983605',
'jeetu_Rajput__20_1700896015',
'Vikas_singh_20',
'sourabh_porwal_20',
'Sumit_kashyap_20',
'Rahul_9205549489_020',
'Manoj__13',
'Bambam PS',
'Rahul kapoor PS',
'Mohd Gulab',
'Sagar bhan RD',
'Ram Autar Rapydex',
'Kamal Jain',
'Sachin Bhan RD'
)
'''

In [3]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('No',path)
df_block = pd.read_sql(query,conn)
conn.close()

Connection Made


NameError: name 'query' is not defined

In [5]:
path = 'tunnel-ssh.cer'
manual_block = """
select 
    rider_id,
    node_node_id,
    rider_name,
    node_name as hub,
    actual_city_name as city,
    locus_rider_id
from application_db.rider as r 
left join application_db.node as n on r.node_node_id = n.node_id
where rider_id in (6609,6611,6605)
"""
conn = get_conn('No',path)
df_block = pd.read_sql(manual_block,conn)
conn.close()

Connection Made


/var/folders/jj/w1s7wr2s7k1b9_vrfwty09pw0000gn/T/ipykernel_34988/2595546501.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_block = pd.read_sql(manual_block,conn)


In [6]:
df_block

,rider_id,node_node_id,rider_name,hub,city,locus_rider_id
0,6605,35,Govind,YLHK-Franchise,Bangalore,Govind_8431763889_035
1,6609,24,rohit raikwar,DS DEL HKMN,Delhi,rohit_9865435128_024
2,6611,40,javed khan,DS DEL GOKP,NCR,javed_9643422770_040


Initiating Blocking

In [7]:
# Configure logging
logging.basicConfig(level=logging.INFO)

# Constants
BASE_URL = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9001'
API_ENDPOINT = '/rider/delist'
HEADERS_BASE = {
    'name': 'void',  # Change this value if needed
    'Content-Type': 'application/json',
}

# Create a session for making multiple requests
session = requests.Session()

# List to store rider IDs with timeout
timeout_rider_ids = []

for index, row in df_block.iterrows():
    headers = HEADERS_BASE.copy()
    headers['node_id'] = str(row['node_node_id'])

    data = {
        'delistType': 'BLACKLIST',
        'riderIds': [str(row['rider_id'])],
    }

    url = f"{BASE_URL}{API_ENDPOINT}"

    try:
        response = session.put(url, headers=headers, json=data, timeout=5)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        json_response = response.json()
        message = json_response.get('message', '')
        logging.info(f"Request for rider_id {row['rider_id']} - {message}")

        data = json_response.get('data', [])
        for item in data:
            rider_id = item.get('riderId', '')
            rider_name = item.get('riderName', '')
            error_message = item.get('message', '')
            logging.info(f"    Rider ID: {rider_id}, Rider Name: {rider_name}, Error Message: {error_message}")

    except requests.exceptions.Timeout:
        logging.error(f"Timeout occurred for rider_id {row['rider_id']}")
        timeout_rider_ids.append(row['rider_id'])

    except requests.exceptions.HTTPError as errh:
        logging.error(f"HTTP Error: {errh}")
    except requests.exceptions.RequestException as err:
        logging.error(f"Request Exception: {err}")

print("Rider blocking completed updating database")

# Print or use timeout_rider_ids as needed
print("Rider IDs with timeout:", timeout_rider_ids)


INFO:root:Request for rider_id 6605 - Success
INFO:root:Request for rider_id 6609 - Success
INFO:root:Request for rider_id 6611 - Success


Rider blocking completed updating database
Rider IDs with timeout: []


Updating Tables

In [10]:
conn = get_conn('No',path)
cursor = conn.cursor()
update_query1 = '''
    UPDATE rider_pip
    SET pip_status = 'final'
    WHERE pip_status = 'closed';
'''
update_query2 = '''
delete from rider_pip where pip_status = 'running' and rider_id in 
(select rider_id from rider_pip where pip_status = 'final' and pip_result = 'failed')
'''
cursor.execute(update_query1)
print("Updated all closed status to final")
cursor.execute(update_query2)
print("Deleted blocked riders from running pip")
conn.commit()
cursor.close()
conn.close()

Connection Made
Updated all closed status to final
Deleted blocked riders from running pip


Sending Closure Mails

In [11]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime

subject = f"Rider PIP Blocked Riders Report - {datetime.today().strftime('%d/%m/%Y')}"
from_email = "akshay@blitznow.in"  # Replace with your email
to_email = "akshay@blitznow.in, sanjay@blitznow.in, david.kumar@blitznow.in, ajay.verma@blitznow.in, biswojit.jena@blitznow.in, chandrakanth.sunkoji@blitznow.in, rahul.yadav@blitznow.in, rahul.lochav@blitznow.in, sourabh.vijay@blitznow.in, sourabh.vijayvergia@blitznow.in, syed.afrid@blitznow.in, anandbellary.hindushree@blitznow.in, dilip.kumar@blitznow.in, ma.saifullah@blitznow.in, shankar.poddar@blitznow.in, sumit.kamble@blitznow.in, anil@blitznow.in"


# Group the DataFrame by 'city' and 'hub' and format the data
blocked_riders_by_city_hub = df_block.groupby(['city', 'hub'])
email_content = "<p>Hi Team, These are the riders blocked tonight</p>"

for (city, hub), group in blocked_riders_by_city_hub:
    email_content += f"<p><b>City:</b> {city} <b>Hub:</b> {hub}</p>"
    email_content += group[['locus_rider_id', 'rider_name']].to_html(index=False, justify='center', classes='table', escape=False)

# Add inline styles for the table header
email_content = email_content.replace('<th>', '<th style="background-color: #c7dceb;">')

# Compose the email
msg = MIMEMultipart()
msg['From'] = from_email
msg['To'] = to_email
msg['Subject'] = subject

# Attach the email content
msg.attach(MIMEText(email_content, 'html'))

# Connect to the SMTP server and send the email
try:
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(from_email, 'Outofblitz1!')  # Replace with your email password
        server.sendmail(from_email, to_email, msg.as_string())
    print("Email sent successfully.")
except Exception as e:
    print(f"Error sending email: {e}")


Email sent successfully.
